<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Implementing Q-Learning in Keras**


Estimated time needed: **30** minutes


## Lab Overview 
In this lab, you will implement a Q-Learning algorithm using Keras to solve a reinforcement learning problem.

## Learning objectives:
By the end of this lab, you will:  
- Implement a Q-Learning algorithm using Keras
- Define and train a neural network to approximate the Q-values
- Evaluate the performance of the trained Q-Learning agent

## Prerequisites 
- Basic knowledge of Python programming
- Familiarity with Keras and neural networks
- Understanding of reinforcement learning concepts


### Step-by-Step Guide 

#### Step 1: Setting Up the Environment 

First, you will set up the environment using the OpenAI Gym library. You will use the 'CartPole-v1' environment, a common benchmark for reinforcement learning algorithms.


In [1]:
%pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 29.6 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 153.0 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827695 sha256=2ae8bdc8ef4f81fc460b23e048c3f78825de77bf13322398cc088587d0bb1dfc
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade numpy==1.26.4
!pip uninstall tensorflow -y 
!pip install tensorflow==2.16.2 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 176.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.8/590.8 MB 20.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 140.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 48.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 57.6 MB/s eta 0:00:00


### Set Environment Variables 
Sometimes, environment variables can help mitigate certain issues with TensorFlow. You can try disabling the oneDNN optimizations or CUDA.


In [3]:
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0' 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' 

### Reduce Recursion Limit 
You can also try increasing the recursion limit, although this is generally more of a workaround than a solution.


In [4]:
import sys 
sys.setrecursionlimit(1500) 

import gym 
import numpy as np 

# Create the environment 
env = gym.make('CartPole-v1') 

# Set random seed for reproducibility 
np.random.seed(42) 
env.action_space.seed(42) 
env.observation_space.seed(42)

[42]

#### Explanation:  
- `gym` is a toolkit for developing and comparing reinforcement learning algorithms.
- `CartPole-v1` is an environment where a pole is balanced on a cart, and the goal is to prevent the pole from falling over.
- Setting random seeds ensures that you can reproduce the results.


### Step 2: Define the Q-Learning Model 

You will define a neural network using Keras to approximate the Q-values. The network will take the state as input and output Q-values for each action.


In [5]:
# Suppress warnings for a cleaner notebook or console experience
import warnings
warnings.filterwarnings('ignore')

# Override the default warning function
def warn(*args, **kwargs):
    pass
warnings.warn = warn

# Import necessary libraries for the Q-Learning model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input  # Import Input layer
from tensorflow.keras.optimizers import Adam
import gym  # Ensure the environment library is available

# Define the model building function
def build_model(state_size, action_size): 
    model = Sequential() 
    model.add(Input(shape=(state_size,)))  # Use Input layer to specify the input shape 
    model.add(Dense(24, activation='relu')) 
    model.add(Dense(24, activation='relu')) 
    model.add(Dense(action_size, activation='linear')) 
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001)) 
    return model 

# Create the environment and set up the model
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0] 
action_size = env.action_space.n 
model = build_model(state_size, action_size)


2025-01-22 06:26:09.282250: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 06:26:09.286976: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 06:26:09.299950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 06:26:09.328937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 06:26:09.329014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 06:26:09.344148: I tensorflow/core/platform/cpu_feature_guard.cc:

#### Explanation: 
- `Sequential` model: a linear stack of layers in Keras. 
- `Dense` layers: fully connected layers. 
- `input_dim`: the size of the input layer, corresponding to the state size. 
- `activation='relu'`: Rectified Linear Unit activation function. 
- `activation='linear'`: linear activation function for the output layer, as we are predicting continuous Q-values. 
- `Adam` optimizer: an optimization algorithm that adjusts the learning rate based on gradients. 


#### Step 3: Implement the Q-Learning Algorithm 

Now, you will implement the Q-Learning algorithm, which involves interacting with the environment, updating the Q-values, and training the neural network. 

**Define the replay Function**


In [6]:
import random
import numpy as np
from collections import deque
import tensorflow as tf

# Define epsilon and epsilon_decay
epsilon = 1.0  # Starting with a high exploration rate
epsilon_min = 0.01  # Minimum exploration rate
epsilon_decay = 0.99  # Faster decay rate for epsilon after each episode

# Replay memory
memory = deque(maxlen=2000)

def remember(state, action, reward, next_state, done):
    """Store experience in memory."""
    memory.append((state, action, reward, next_state, done))

def replay(batch_size=64):  # Increased batch size
    """Train the model using a random sample of experiences from memory."""
    if len(memory) < batch_size:
        return  # Skip replay if there's not enough experience

    minibatch = random.sample(memory, batch_size)  # Sample a random batch from memory
    
    # Extract information for batch processing
    states = np.vstack([x[0] for x in minibatch])
    actions = np.array([x[1] for x in minibatch])
    rewards = np.array([x[2] for x in minibatch])
    next_states = np.vstack([x[3] for x in minibatch])
    dones = np.array([x[4] for x in minibatch])
    
    # Predict Q-values for the next states in batch
    q_next = model.predict(next_states)
    # Predict Q-values for the current states in batch
    q_target = model.predict(states)
    
    # Vectorized update of target values
    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += 0.95 * np.amax(q_next[i])  # Update Q value with the discounted future reward
        q_target[i][actions[i]] = target  # Update only the taken action's Q value
    
    # Train the model with the updated targets in batch
    model.fit(states, q_target, epochs=1, verbose=0)  # Train in batch mode

    # Reduce exploration rate (epsilon) after each training step
    global epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

def act(state):
    """Choose an action based on the current state and exploration rate."""
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)  # Explore: choose a random action
    act_values = model.predict(state)  # Exploit: predict action based on the state
    return np.argmax(act_values[0])  # Return the action with the highest Q-value

# Define the number of episodes you want to train the model for
episodes = 10  # You can set this to any number you prefer
train_frequency = 5  # Train the model every 5 steps

for e in range(episodes):
    state, _ = env.reset()  # Unpack the tuple returned by env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(200):  # Limit to 200 time steps per episode
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)  # Store experience
        state = next_state
        
        if done:
            print(f"episode: {e+1}/{episodes}, score: {time}, e: {epsilon:.2}")
            break
        
        # Train the model every 'train_frequency' steps
        if time % train_frequency == 0:
            replay(batch_size=64)  # Call replay with larger batch size for efficiency

env.close()


episode: 1/10, score: 16, e: 1.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
episode: 2/10, score: 84, e: 0.93
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
episode: 3/10, score: 9, e: 0.91
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

#### Step 4: Evaluate the Performance 

Finally, you will evaluate the performance of the trained Q-Learning agent.


In [7]:
for e in range(10):  

    state, _ = env.reset()  # Unpack the state from the tuple 
    state = np.reshape(state, [1, state_size])  # Reshape the state correctly 
    for time in range(500):  
        env.render()  
        action = np.argmax(model.predict(state)[0])  
        next_state, reward, terminated, truncated, _ = env.step(action)  # Unpack the five return values 
        done = terminated or truncated  # Check if the episode is done 
        next_state = np.reshape(next_state, [1, state_size])  
        state = next_state  
        if done:  
            print(f"episode: {e+1}/10, score: {time}")  
            break  

env.close() 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
episode: 1/10, score: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
episode: 2/10, score: 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/

#### Explanation: 
- This loop runs 10 episodes to test the trained agent. 
- `env.render()`: visualizes the environment. 
- The agent chooses actions based on the trained model and interacts with the environment. 
- The score for each episode is printed.


# Practice exercises 

## Exercise 1: Experiment with Different Network Architectures 

### Objective: 
Understand how changing the architecture of the neural network affects the performance of the Q-Learning agent. 

### Instructions: 
1. Modify the `build_model()` function to include a different number of neurons and layers. For example, increase the number of layers to 3 and the number of neurons in each layer to 64. 
2. Train the model with the modified architecture and observe the performance in terms of average score achieved over 100 episodes. 
3. Compare the performance with the original architecture.


In [8]:
# Write your code here

def build_model_v2(state_size, action_size):
    """
    Builds a modified neural network for Q-learning with a deeper architecture.
    
    Parameters:
    - state_size: Number of features in the state representation
    - action_size: Number of possible actions
    
    Returns:
    - model: A compiled Keras model
    """
    model = Sequential()
    model.add(Input(shape=(state_size,)))  # Input layer
    model.add(Dense(64, activation='relu'))  # First hidden layer
    model.add(Dense(64, activation='relu'))  # Second hidden layer
    model.add(Dense(64, activation='relu'))  # Third hidden layer
    model.add(Dense(action_size, activation='linear'))  # Output layer
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model


In [9]:
model = build_model_v2(state_size, action_size)


In [15]:
import random
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import gym

# Build the modified Q-network
def build_model_v2(state_size, action_size):
    model = Sequential()
    model.add(Input(shape=(state_size,)))
    model.add(Dense(32, activation='relu'))  # Reduced neurons to optimize memory usage
    model.add(Dense(32, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

# Environment setup
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
model = build_model_v2(state_size, action_size)

# Q-Learning variables
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.99
gamma = 0.95  # Discount factor
batch_size = 32  # Reduced batch size
memory = deque(maxlen=1000)  # Reduced memory size to save RAM

# Helper functions
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def replay():
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    states = np.vstack([x[0] for x in minibatch])
    actions = np.array([x[1] for x in minibatch])
    rewards = np.array([x[2] for x in minibatch])
    next_states = np.vstack([x[3] for x in minibatch])
    dones = np.array([x[4] for x in minibatch])
    
    q_next = model.predict(next_states, verbose=0)
    q_target = model.predict(states, verbose=0)
    
    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += gamma * np.amax(q_next[i])
        q_target[i][actions[i]] = target
    
    model.fit(states, q_target, epochs=1, verbose=0)
    
    global epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

def act(state):
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    act_values = model.predict(state, verbose=0)
    return np.argmax(act_values[0])

# Training loop
episodes = 200  # Increased episodes for better learning
memory = deque(maxlen=5000)  # Increased replay memory size

for e in range(episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(200):  # Limited max steps per episode to 200
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10  # Penalize termination
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"Episode: {e+1}/{episodes}, Score: {time}, Epsilon: {epsilon:.2f}")
            break
    replay()

scores = []
evaluation_episodes = 50  # Increased evaluation episodes for reliability
max_steps = 200  # Limit max steps per evaluation episode

for e in range(evaluation_episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    score = 0
    for time in range(max_steps):
        action = np.argmax(model.predict(state, verbose=0)[0])  # Exploitation only
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state
        score += 1
        if done:
            break
    scores.append(score)
    print(f"Episode: {e+1}/{evaluation_episodes}, Score: {score}")

# Compute average score
average_score = np.mean(scores)
print(f"Average Score over {evaluation_episodes} episodes: {average_score}")


Episode: 1/200, Score: 41, Epsilon: 1.00
Episode: 2/200, Score: 12, Epsilon: 0.99
Episode: 3/200, Score: 16, Epsilon: 0.98
Episode: 4/200, Score: 58, Epsilon: 0.97
Episode: 5/200, Score: 15, Epsilon: 0.96
Episode: 6/200, Score: 36, Epsilon: 0.95
Episode: 7/200, Score: 15, Epsilon: 0.94
Episode: 8/200, Score: 20, Epsilon: 0.93
Episode: 9/200, Score: 24, Epsilon: 0.92
Episode: 10/200, Score: 36, Epsilon: 0.91
Episode: 11/200, Score: 12, Epsilon: 0.90
Episode: 12/200, Score: 27, Epsilon: 0.90
Episode: 13/200, Score: 21, Epsilon: 0.89
Episode: 14/200, Score: 16, Epsilon: 0.88
Episode: 15/200, Score: 17, Epsilon: 0.87
Episode: 16/200, Score: 8, Epsilon: 0.86
Episode: 17/200, Score: 9, Epsilon: 0.85
Episode: 18/200, Score: 13, Epsilon: 0.84
Episode: 19/200, Score: 13, Epsilon: 0.83
Episode: 20/200, Score: 12, Epsilon: 0.83
Episode: 21/200, Score: 19, Epsilon: 0.82
Episode: 22/200, Score: 11, Epsilon: 0.81
Episode: 23/200, Score: 14, Epsilon: 0.80
Episode: 24/200, Score: 20, Epsilon: 0.79
Epi

<details>
    <summary>Click here for Solution</summary>

```python
# Install gym if necessary
!pip install gym

import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from collections import deque
import random

# Initialize the environment
env = gym.make('CartPole-v1')

# Global settings
episodes = 10  # Number of episodes
batch_size = 32  # Size of the mini-batch for training
memory = deque(maxlen=2000)  # Memory buffer to store experiences

# Define state size and action size based on the environment
state_size = env.observation_space.shape[0]  # State space size from the environment
action_size = env.action_space.n  # Number of possible actions from the environment

# Define the model
def build_model(state_size, action_size):
    model = Sequential()
    model.add(Input(shape=(state_size,)))  # Explicit Input layer
    model.add(Dense(32, activation='relu'))  # Smaller hidden layers
    model.add(Dense(32, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

# Re-initialize the model with the new architecture
model = build_model(state_size, action_size)

# Placeholder for your action function (e.g., epsilon-greedy)
def act(state):
    return env.action_space.sample()  # For now, a random action is taken

# Function to remember experiences in memory
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

# Optimized function to replay experiences from memory and train the model
def replay(batch_size):
    minibatch = random.sample(memory, batch_size)
    states = np.vstack([sample[0] for sample in minibatch])
    next_states = np.vstack([sample[3] for sample in minibatch])
    targets = model.predict(states)
    target_next = model.predict(next_states)
    
    for i, (state, action, reward, next_state, done) in enumerate(minibatch):
        target = reward if done else reward + 0.95 * np.amax(target_next[i])
        targets[i][action] = target
        
    model.fit(states, targets, epochs=1, verbose=0)

# Train the model with the modified architecture
for e in range(episodes):
    state, _ = env.reset()  # Unpack the state from the tuple
    state = np.reshape(state, [1, state_size])
    for time in range(200):  # Reduced number of steps per episode
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            print(f"episode: {e+1}/{episodes}, score: {time}")
            break
        
        if len(memory) > batch_size and time % 10 == 0:  # Train every 10 steps
            replay(batch_size)  # Pass the batch size to replay()

env.close()
```

</details>


## Exercise 2: Implement an Adaptive Exploration Rate 

### Objective: 
Learn how to adapt the exploration rate (`epsilon`) based on the agent's performance to balance exploration and exploitation. 

### Instructions: 
1. Modify the `epsilon` decay strategy to decrease more rapidly when the agent's performance improves significantly. 
2. Implement a check to reduce `epsilon` faster if the agent achieves a score greater than a certain threshold (e.g., 200) in consecutive episodes. 
3. Observe the effect on the learning rate and the agent's performance. 


In [16]:
# Write your code here

# Adaptive exploration rate parameters
consecutive_goal_threshold = 200  # Score threshold to trigger faster decay
consecutive_goal_count = 5  # Number of consecutive episodes to check
goal_streak = 0  # Counter for consecutive successful episodes

# Training loop with adaptive epsilon
episodes = 200  # Total number of training episodes
adaptive_epsilon_decay = 0.995  # Faster decay rate for adaptive epsilon

for e in range(episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    score = 0

    for time in range(200):  # Max steps per episode
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)
        state = next_state
        score += 1
        if done:
            break

    # Update goal streak
    if score >= consecutive_goal_threshold:
        goal_streak += 1
    else:
        goal_streak = 0

    # Apply adaptive epsilon decay
    if goal_streak >= consecutive_goal_count:
        epsilon *= adaptive_epsilon_decay
        epsilon = max(epsilon, epsilon_min)  # Ensure epsilon doesn't fall below the minimum
        print(f"Adaptive Epsilon Decay Triggered: New Epsilon: {epsilon:.4f}")
        goal_streak = 0  # Reset the streak after applying decay

    replay()  # Train the model

    print(f"Episode: {e+1}/{episodes}, Score: {score}, Epsilon: {epsilon:.4f}")



Episode: 1/200, Score: 24, Epsilon: 0.1326
Episode: 2/200, Score: 21, Epsilon: 0.1313
Episode: 3/200, Score: 23, Epsilon: 0.1300
Episode: 4/200, Score: 28, Epsilon: 0.1287
Episode: 5/200, Score: 30, Epsilon: 0.1274
Episode: 6/200, Score: 21, Epsilon: 0.1261
Episode: 7/200, Score: 25, Epsilon: 0.1249
Episode: 8/200, Score: 23, Epsilon: 0.1236
Episode: 9/200, Score: 28, Epsilon: 0.1224
Episode: 10/200, Score: 24, Epsilon: 0.1212
Episode: 11/200, Score: 20, Epsilon: 0.1200
Episode: 12/200, Score: 18, Epsilon: 0.1188
Episode: 13/200, Score: 26, Epsilon: 0.1176
Episode: 14/200, Score: 26, Epsilon: 0.1164
Episode: 15/200, Score: 27, Epsilon: 0.1152
Episode: 16/200, Score: 17, Epsilon: 0.1141
Episode: 17/200, Score: 22, Epsilon: 0.1129
Episode: 18/200, Score: 34, Epsilon: 0.1118
Episode: 19/200, Score: 17, Epsilon: 0.1107
Episode: 20/200, Score: 30, Epsilon: 0.1096
Episode: 21/200, Score: 26, Epsilon: 0.1085
Episode: 22/200, Score: 25, Epsilon: 0.1074
Episode: 23/200, Score: 23, Epsilon: 0.10

<details>
    <summary>Click here for Solution</summary>

```python
# Function to adjust epsilon based on performance
def adjust_epsilon(score, consecutive_success_threshold=200):
    global epsilon 

    if score >= consecutive_success_threshold: 
        epsilon = max(epsilon_min, epsilon * 0.9)  # Reduce epsilon faster if performance is good
    else: 
        epsilon = max(epsilon_min, epsilon * epsilon_decay)  # Regular epsilon decay

episodes = 2  # Set number of episodes for training

# Train the model with adaptive epsilon decay
for e in range(episodes): 
    state = env.reset()  
    state = state[0]  # Extract the first element, which is the actual state array
    state = np.reshape(state, [1, len(state)])  # Reshape state to match the expected input shape

    total_reward = 0 

    for time in range(500):  # Limit the episode to 500 time steps
        action = act(state)  # Choose action based on policy
        next_state, reward, done, truncated, _ = env.step(action)  # Unpack 5 values

        reward = reward if not done else -10  # Penalize for reaching a terminal state
        total_reward += reward  # Accumulate rewards

        next_state = np.reshape(next_state, [1, len(next_state)])  # Reshape next state (optional based on model needs)

        remember(state, action, reward, next_state, done)  # Store experience in memory
        state = next_state  # Update the current state

        if done or truncated:  # Check if the episode is done or truncated
            adjust_epsilon(total_reward)  # Adjust epsilon based on the total reward
            print(f"episode: {e}/{episodes}, score: {time}, e: {epsilon:.2}")  # Print the episode details
            break  # Break out of the loop if the episode is done or truncated

        if len(memory) > batch_size:  # Check if enough experiences are stored in memory
            replay(batch_size)  # Train the model with the stored experiences (pass batch_size here)
```

</details>


## Exercise 3 : Implement a Custom Reward Function 

### Objective: 
Understand the impact of reward shaping on training the Q-Learning agent. 

### Instructions: 
1. Modify the reward function to provide more granular feedback to the agent. For example, give higher rewards for keeping the pole more vertical and closer to the center. 
2. Implement a reward function that rewards the agent proportionally to the angle of the pole and the distance of the cart from the center. 
3. Train the agent with the new reward function and compare the learning speed and stability to the original setting.


In [17]:
# Write your code here

# Custom reward function
def calculate_custom_reward(state, terminated):
    """
    Calculate a custom reward based on the state and termination status.

    Args:
    - state: Current state of the environment [x, x_dot, theta, theta_dot]
    - terminated: Boolean indicating if the episode ended

    Returns:
    - reward: Custom reward value
    """
    x, x_dot, theta, theta_dot = state
    pole_angle_penalty = abs(theta)  # Penalize deviation of pole angle
    cart_position_penalty = abs(x)  # Penalize deviation from the center

    # Combine penalties into a reward
    reward = 1.0 - (0.1 * pole_angle_penalty + 0.1 * cart_position_penalty)

    # Apply heavy penalty for termination
    if terminated:
        reward -= 10

    return reward



In [18]:
# Training loop with custom reward function
for e in range(episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    score = 0

    for time in range(200):  # Max steps per episode
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        # Calculate custom reward
        custom_reward = calculate_custom_reward(next_state, terminated)
        
        done = terminated or truncated
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, custom_reward, next_state, done)
        state = next_state
        score += 1
        
        if done:
            break

    replay()
    print(f"Episode: {e+1}/{episodes}, Score: {score}, Epsilon: {epsilon:.4f}")


Episode: 1/200, Score: 20, Epsilon: 0.0178
Episode: 2/200, Score: 15, Epsilon: 0.0176
Episode: 3/200, Score: 18, Epsilon: 0.0174
Episode: 4/200, Score: 22, Epsilon: 0.0172
Episode: 5/200, Score: 22, Epsilon: 0.0171
Episode: 6/200, Score: 15, Epsilon: 0.0169
Episode: 7/200, Score: 23, Epsilon: 0.0167
Episode: 8/200, Score: 17, Epsilon: 0.0166
Episode: 9/200, Score: 24, Epsilon: 0.0164
Episode: 10/200, Score: 17, Epsilon: 0.0162
Episode: 11/200, Score: 28, Epsilon: 0.0161
Episode: 12/200, Score: 21, Epsilon: 0.0159
Episode: 13/200, Score: 16, Epsilon: 0.0158
Episode: 14/200, Score: 19, Epsilon: 0.0156
Episode: 15/200, Score: 14, Epsilon: 0.0154
Episode: 16/200, Score: 18, Epsilon: 0.0153
Episode: 17/200, Score: 13, Epsilon: 0.0151
Episode: 18/200, Score: 18, Epsilon: 0.0150
Episode: 19/200, Score: 15, Epsilon: 0.0148
Episode: 20/200, Score: 17, Epsilon: 0.0147
Episode: 21/200, Score: 16, Epsilon: 0.0145
Episode: 22/200, Score: 18, Epsilon: 0.0144
Episode: 23/200, Score: 20, Epsilon: 0.01

<details>
    <summary>Click here for Solution</summary>

```python
# Define a custom reward function based on the cart position and pole angle
def custom_reward(state):
    # Extract state variables: x (cart position), x_dot (cart velocity), theta (pole angle), theta_dot (pole angular velocity)
    x, x_dot, theta, theta_dot = state
    
    # Custom reward function: Encourage the agent to keep the cart near the center and the pole upright
    reward = (1 - abs(x) / 2.4) + (1 - abs(theta) / 0.20948)
    
    return reward

episodes = 2  # Number of episodes to run

# Train the model with the custom reward function
for e in range(episodes): 
    state = env.reset()  # Reset the environment

    # Print the state structure for debugging
    print(f"State: {state}, State Type: {type(state)}")

    # Extract the state if it's a tuple and reshape if necessary
    if isinstance(state, tuple):
        state = state[0]  # Extract the first element if it's a tuple

    state = np.reshape(state, [1, state_size])  # Reshape state to match the expected input shape

    for time in range(500):  # Limit the episode to 500 time steps
        action = act(state)  # Choose an action based on the current state
        
        # Unpack 5 values returned by env.step(action)
        next_state, reward, done, truncated, _ = env.step(action)

        # Compute the custom reward based on the next state
        reward = custom_reward(next_state) if not done else -10

        # Reshape next_state if necessary
        if isinstance(next_state, tuple):
            next_state = next_state[0]  # Extract the first element if it's a tuple

        next_state = np.reshape(next_state, [1, state_size])  # Reshape next state to match input shape

        # Store the experience in memory
        remember(state, action, reward, next_state, done)
        state = next_state  # Update the current state

        if done or truncated:  # If the episode is done, break out of the loop
            print(f"episode: {e}/{episodes}, score: {time}, e: {epsilon:.2}")
            break

        if len(memory) > batch_size:  # If there are enough samples in memory, train the model
            replay(batch_size)  # Train the model with a batch of experiences

```

</details>


#### Conclusion 

Congratulations on completing this lab!  In this lab, you explored various strategies to enhance the performance of the Q-Learning agent, such as experimenting with different network architectures, implementing adaptive exploration rates, and customizing the reward function. These variations help reinforce your understanding of the Q-Learning algorithm's flexibility and the impact of different hyperparameters and strategies on the learning process.  


## Authors


Skills Network


Copyright © IBM Corporation. All rights reserved.
